# Euro Area Bond Yield Spillover Analysis
## Diebold-Yilmaz (2012) Generalized FEVD Framework

**What this notebook does:**
- Loads daily 10-year sovereign bond yield data for 7 countries (US, Japan, UK, Germany, France, Italy, Spain)
- Estimates a Vector Autoregression (VAR) model on yield *changes*
- Computes **Generalized Forecast Error Variance Decomposition** (GFEVD) — an order-invariant method
- Rolls this estimation through time to produce time-varying spillover measures
- Generates charts and tables summarising who affects whom in global bond markets

**Key references:**
- Diebold & Yilmaz (2009, 2012): Spillover index methodology
- Pesaran & Shin (1998): Generalized impulse responses and FEVD (order-invariant)
- Klössner & Wagner (2014): Exploration of all VAR orderings (motivates the Generalized approach)

**How to re-run with different settings:** Change the parameters in Section 2 and re-run all cells.

## 1. Setup & Package Installation

The cell below installs any missing packages. If you're on Cloudera, you may need to run this once.
After that, it imports everything we need.

| Package | Purpose |
|---------|---------|
| `pandas` | Data manipulation (DataFrames) |
| `numpy` | Numerical computation (matrix algebra for FEVD) |
| `matplotlib` | Plotting charts |
| `statsmodels` | VAR model estimation |
| `scipy` | Statistical tests |
| `openpyxl` | Excel file export |

In [ ]:
# Install missing packages (run once, then comment out if needed)
import subprocess, sys
for pkg in ['pandas', 'numpy', 'matplotlib', 'statsmodels', 'scipy', 'openpyxl']:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (12, 5)

print("All packages loaded successfully.")

## 2. Configuration — Parameters You Can Change

**All tuneable parameters are in the cell below.** Change them here and re-run the notebook.

| Parameter | Default | What it controls | Typical range |
|-----------|---------|-----------------|---------------|
| `VAR_LAGS` | 4 | Number of lags in the VAR model | 2–6 for daily data |
| `FORECAST_HORIZON` | 10 | Days ahead for variance decomposition | 5–20 |
| `ROLLING_WINDOW` | 200 | Trading days per rolling window (~10 months) | 100–500 |
| `STEP_SIZE` | 5 | Days between successive windows (speed vs smoothness) | 1–10 |
| `DATA_PATH` | `nav_test_clean.csv` | Input CSV file path | Any CSV with same column format |

**Country lists** — you can add or remove countries if your CSV has more columns:
- `EXTERNAL`: Countries outside the Euro Area
- `EA_CORE`: Core EA members
- `EA_PERIPHERY`: Peripheral EA members

In [ ]:
# =====================================================================
# PARAMETERS — CHANGE THESE TO MODIFY THE ANALYSIS
# =====================================================================

VAR_LAGS = 4            # Lags in the VAR. Higher = longer memory, more data needed per window.
FORECAST_HORIZON = 10   # Steps ahead for FEVD. 10 = two trading weeks.
ROLLING_WINDOW = 200    # Trading days per window. 200 ≈ 10 months.
STEP_SIZE = 5           # Advance between windows. 1 = daily (slow), 5 = weekly (fast).

DATA_PATH = "nav_test_clean.csv"  # Relative to notebook location, or use absolute path

# Country groupings — must match column names in the CSV
EXTERNAL     = ["US", "JP", "UK"]
EA_CORE      = ["DE", "FR"]
EA_PERIPHERY = ["IT", "ES"]

# Derived (don't change unless you change the above)
EA = EA_CORE + EA_PERIPHERY
ALL_COUNTRIES = EXTERNAL + EA

# Colour scheme for charts (one colour per country)
COLORS = {
    "US": "#003299", "JP": "#FFB400", "UK": "#FF4B00",
    "DE": "#65B800", "FR": "#00B1EA", "IT": "#007816", "ES": "#8139C6"
}

print(f"Analysis covers {len(ALL_COUNTRIES)} countries: {ALL_COUNTRIES}")
print(f"VAR({VAR_LAGS}), {FORECAST_HORIZON}-day horizon, {ROLLING_WINDOW}-day rolling window, step={STEP_SIZE}")

## 3. Load & Explore the Data

We load a CSV of **daily 10-year government bond yields** (in percent).
Each row is a trading day. Columns are country ISO codes.

The data runs from **January 1995 to ~2025**, giving us roughly 30 years of daily observations.

In [ ]:
# Load the yield data
yields = pd.read_csv(DATA_PATH)
yields['Date'] = pd.to_datetime(yields['Date'])
yields = yields.sort_values('Date').reset_index(drop=True)

# Keep only the columns we need
cols_to_keep = ['Date'] + [c for c in ALL_COUNTRIES if c in yields.columns]
yields = yields[cols_to_keep]

# Convert to numeric (handles any stray text)
for c in ALL_COUNTRIES:
    yields[c] = pd.to_numeric(yields[c], errors='coerce')

print(f"Loaded {len(yields):,} daily observations")
print(f"Date range: {yields['Date'].min().date()} to {yields['Date'].max().date()}")
print(f"\nColumns: {list(yields.columns)}")
print(f"\nMissing values per country:")
print(yields[ALL_COUNTRIES].isnull().sum())
print(f"\nFirst 5 rows:")
yields.head()

In [ ]:
# Plot raw yield levels
fig, ax = plt.subplots(figsize=(14, 6))
for c in ALL_COUNTRIES:
    ax.plot(yields['Date'], yields[c], label=c, color=COLORS[c], linewidth=0.8)
ax.set_xlabel('Date')
ax.set_ylabel('10-Year Government Bond Yield (%)')
ax.set_title('Raw Bond Yields: All Countries (1995–2025)', fontweight='bold')
ax.legend(ncol=4, loc='upper right')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Data Preparation — Why We Use Yield *Changes*

**Critical assumption:** VAR models require (approximately) stationary data.
Bond yield *levels* are non-stationary (they trend and have unit roots).
Yield *changes* (first differences: today's yield minus yesterday's) are stationary.

$$\Delta y_t = y_t - y_{t-1}$$

**If you used levels instead**, the VAR would produce unreliable results — spurious
relationships, inflated t-statistics, and meaningless spillover estimates.

The cell below computes daily yield changes and drops the first row (which becomes NaN).

In [ ]:
# Compute first differences (yield changes)
yield_changes = yields.copy()
for c in ALL_COUNTRIES:
    yield_changes[c] = yields[c].diff()

# Drop the first row (NaN from differencing)
yield_changes = yield_changes.dropna(subset=ALL_COUNTRIES).reset_index(drop=True)

print(f"Yield changes: {len(yield_changes):,} observations")
print(f"\nSummary statistics of daily yield changes (basis points × 100):")
print(yield_changes[ALL_COUNTRIES].describe().round(4))

In [ ]:
# Plot yield changes
fig, axes = plt.subplots(len(ALL_COUNTRIES), 1, figsize=(14, 2*len(ALL_COUNTRIES)), sharex=True)
for i, c in enumerate(ALL_COUNTRIES):
    axes[i].plot(yield_changes['Date'], yield_changes[c], color=COLORS[c], linewidth=0.3)
    axes[i].set_ylabel(c, fontsize=10, fontweight='bold')
    axes[i].grid(alpha=0.2)
    axes[i].axhline(0, color='black', linewidth=0.3)
axes[-1].set_xlabel('Date')
fig.suptitle('Daily Yield Changes by Country', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

## 5. Methodology: Generalized Forecast Error Variance Decomposition

This is the most important section to understand. Read carefully.

### What is a VAR model?

A **Vector Autoregression (VAR)** is a system of equations where each variable depends on
its own past values AND the past values of all other variables:

$$\Delta y_t = A_1 \Delta y_{t-1} + A_2 \Delta y_{t-2} + ... + A_p \Delta y_{t-p} + u_t$$

where:
- $\Delta y_t$ is a vector of yield changes for all 7 countries on day $t$
- $A_1, ..., A_p$ are coefficient matrices (estimated from data)
- $p$ = number of lags (our `VAR_LAGS` parameter)
- $u_t$ = residual shocks (the "surprise" component)

### What is Forecast Error Variance Decomposition (FEVD)?

Once we estimate the VAR, we can ask: **"If I forecast German yields 10 days ahead,
how much of my forecast error is due to shocks from the US? From France? From Italy?"**

The FEVD matrix $\Theta$ is a $k \times k$ matrix where:
- $\Theta_{ij}$ = percentage of country $i$'s forecast error variance explained by shocks from country $j$
- Each row sums to 100%
- Diagonal elements = "own" effects (how much of Germany's variance is from German shocks)
- Off-diagonal elements = **spillovers** (how much of Germany's variance is from *other* countries' shocks)

### Why "Generalized"?

The traditional (Cholesky) FEVD **depends on the ordering of variables**. If you put the US
first, you get different results than if you put Germany first. This is arbitrary and problematic.

The **Generalized FEVD** (Pesaran & Shin, 1998) solves this by using each variable's own shock
as the conditioning variable. The result is **order-invariant** — you get the same answer
regardless of how you order the countries.

*Trade-off:* Generalized FEVD rows don't naturally sum to 100%, so we normalise them.
This is standard practice (Diebold & Yilmaz, 2012).

### How Spillover Indices are Derived

From the FEVD matrix $\Theta$, we compute:

| Measure | Formula | Interpretation |
|---------|---------|---------------|
| **Total Spillover Index** | $\frac{\sum_{i \neq j} \Theta_{ij}}{k}$ | Overall interconnectedness (0–100%). Higher = more integrated. |
| **TO others** (country $j$) | $\sum_{i \neq j} \Theta_{ij}$ (column $j$ sum minus diagonal) | How much country $j$ transmits to all others combined |
| **FROM others** (country $i$) | $\sum_{j \neq i} \Theta_{ij}$ (row $i$ sum minus diagonal) | How much country $i$ receives from all others combined |
| **Net** (country $j$) | TO $-$ FROM | Positive = **net transmitter**, Negative = **net receiver** |
| **Bilateral** $\Theta_{ij}$ | Single cell | How much of country $i$'s variance comes from country $j$'s shocks |

## 6. Core Functions

The three functions below implement the entire spillover methodology.
You should not need to change these unless you want to modify the methodology itself.

In [ ]:
def generalized_fevd(var_result, H=10):
    """
    Compute Generalized Forecast Error Variance Decomposition.
    Based on Pesaran & Shin (1998), as used in Diebold-Yilmaz (2012).
    This is ORDER-INVARIANT — results don't depend on variable ordering.
    
    Parameters
    ----------
    var_result : statsmodels VARResults object (from model.fit())
    H : int
        Forecast horizon in days. Default 10 (2 trading weeks).
    
    Returns
    -------
    theta_normalized : numpy array (k × k)
        GFEVD matrix. theta[i,j] = % of country i's forecast error variance
        explained by shocks from country j. Rows sum to 100.
    """
    k = var_result.neqs       # Number of variables (countries)
    p = var_result.k_ar       # Number of VAR lags
    A = var_result.coefs      # Coefficient matrices, shape (p, k, k)
    
    # Residual covariance matrix Σ
    sigma = var_result.sigma_u
    if hasattr(sigma, 'values'):
        sigma = sigma.values   # statsmodels sometimes returns a DataFrame
    sigma_diag = np.diag(sigma)  # Diagonal = each variable's residual variance (σ²_jj)
    
    # --- Step 1: Moving Average (MA) representation ---
    # The VAR can be rewritten as an infinite MA process:
    #   Δy_t = Φ_0 u_t + Φ_1 u_{t-1} + Φ_2 u_{t-2} + ...
    # where Φ_0 = I (identity), and Φ_s is computed recursively.
    Phi = np.zeros((H + 1, k, k))
    Phi[0] = np.eye(k)
    
    for s in range(1, H + 1):
        for j in range(1, min(s, p) + 1):
            Phi[s] += Phi[s - j] @ A[j - 1]
    
    # --- Step 2: Generalized FEVD ---
    # For each (i, j) pair, compute what fraction of i's H-step forecast error
    # variance is attributable to shocks from j.
    #
    # Formula: θ_ij = [σ_jj^{-1} × Σ_{h=0}^{H-1} (e_i' Φ_h Σ e_j)²]
    #                 / [Σ_{h=0}^{H-1} (e_i' Φ_h Σ Φ_h' e_i)]
    theta = np.zeros((k, k))
    
    for i in range(k):
        # Denominator: TOTAL forecast error variance for variable i at horizon H
        denom = 0
        for h in range(H):
            denom += Phi[h][i, :] @ sigma @ Phi[h][i, :].T
        
        for j in range(k):
            # Numerator: contribution of shock j to variable i's forecast error
            numer = 0
            for h in range(H):
                numer += (Phi[h][i, :] @ sigma[:, j]) ** 2
            
            theta[i, j] = (numer / sigma_diag[j]) / denom if denom > 0 else 0
    
    # --- Step 3: Normalise ---
    # Generalized FEVD rows don't sum to 1 (unlike Cholesky).
    # We normalise each row to sum to 100%, following Diebold-Yilmaz (2012).
    row_sums = theta.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    theta_normalized = 100 * theta / row_sums
    
    return theta_normalized

print("generalized_fevd() defined.")

In [ ]:
def compute_spillover_indices(theta, var_names):
    """
    From a GFEVD matrix, compute all the Diebold-Yilmaz spillover measures.
    
    Parameters
    ----------
    theta : numpy array (k × k), GFEVD matrix (rows sum to 100)
    var_names : list of str, country names matching theta rows/columns
    
    Returns
    -------
    dict with keys:
        'total_spillover' : float (0–100)
        'to_others'       : dict {country: value}  — how much each transmits
        'from_others'     : dict {country: value}  — how much each receives
        'net'             : dict {country: value}  — to minus from
        'pairwise'        : dict {f'{j}_to_{i}': value}  — bilateral spillovers
        'theta'           : the original GFEVD matrix
    """
    k = len(var_names)
    
    # Total Spillover Index = sum of off-diagonal / k
    off_diag_sum = theta.sum() - np.trace(theta)
    total_spillover = off_diag_sum / k
    
    # Directional TO others = column sum minus diagonal
    to_others = theta.sum(axis=0) - np.diag(theta)
    
    # Directional FROM others = row sum minus diagonal
    from_others = theta.sum(axis=1) - np.diag(theta)
    
    # Net = TO - FROM (positive = net transmitter)
    net = to_others - from_others
    
    # Bilateral (all i≠j pairs)
    pairwise = {}
    for i, name_i in enumerate(var_names):
        for j, name_j in enumerate(var_names):
            if i != j:
                pairwise[f"{name_j}_to_{name_i}"] = theta[i, j]
    
    return {
        'total_spillover': total_spillover,
        'to_others': dict(zip(var_names, to_others)),
        'from_others': dict(zip(var_names, from_others)),
        'net': dict(zip(var_names, net)),
        'pairwise': pairwise,
        'theta': theta
    }

print("compute_spillover_indices() defined.")

In [ ]:
def rolling_spillover_analysis(data, var_names, window=200, lag=4, horizon=10, step=5):
    """
    Run the spillover analysis in rolling windows through time.
    
    For each window of `window` days, we:
    1. Estimate a VAR(lag) on the yield changes in that window
    2. Compute the Generalized FEVD
    3. Extract all spillover indices
    4. Move forward by `step` days and repeat
    
    Parameters
    ----------
    data     : DataFrame with 'Date' column and country yield change columns
    var_names: list of country column names
    window   : int, rolling window size in trading days
    lag      : int, VAR lag order
    horizon  : int, FEVD forecast horizon
    step     : int, days to advance between windows
    
    Returns
    -------
    DataFrame with one row per window, containing all spillover measures
    """
    results = []
    n = len(data)
    total_windows = (n - window) // step + 1
    
    for i, start in enumerate(range(0, n - window, step)):
        if i % 100 == 0:
            print(f"  Window {i+1}/{total_windows} ({100*i/total_windows:.0f}%)")
        
        end = start + window
        window_data = data.iloc[start:end][var_names].dropna()
        
        # Require at least 90% non-missing data in the window
        if len(window_data) < window * 0.9:
            continue
        
        try:
            model = VAR(window_data)
            var_result = model.fit(lag)
            theta = generalized_fevd(var_result, H=horizon)
            indices = compute_spillover_indices(theta, var_names)
            
            # Store the date of the LAST observation in the window
            result = {
                'Date': data.iloc[end - 1]['Date'],
                'total_spillover': indices['total_spillover']
            }
            
            # Directional measures per country
            for name in var_names:
                result[f'{name}_to'] = indices['to_others'][name]
                result[f'{name}_from'] = indices['from_others'][name]
                result[f'{name}_net'] = indices['net'][name]
            
            # All bilateral pairs
            for key, val in indices['pairwise'].items():
                result[key] = val
            
            results.append(result)
        except:
            continue  # Skip windows where VAR fails (e.g., singular matrix)
    
    print(f"  Completed: {len(results)} windows estimated.")
    return pd.DataFrame(results)

print("rolling_spillover_analysis() defined.")

## 7. Run the Rolling Spillover Analysis

**This is the main computation.** It takes a few minutes depending on your `STEP_SIZE`:
- `STEP_SIZE=1` → ~1,500 windows, takes ~10 minutes
- `STEP_SIZE=5` → ~300 windows, takes ~2 minutes  ← default

The progress counter prints every 100 windows so you can track it.

In [ ]:
%%time

print(f"Running rolling spillover analysis...")
print(f"  VAR({VAR_LAGS}), horizon={FORECAST_HORIZON}, window={ROLLING_WINDOW}, step={STEP_SIZE}")
print(f"  Data: {len(yield_changes):,} observations of yield changes\n")

roll_df = rolling_spillover_analysis(
    yield_changes,
    ALL_COUNTRIES,
    window=ROLLING_WINDOW,
    lag=VAR_LAGS,
    horizon=FORECAST_HORIZON,
    step=STEP_SIZE
)

roll_df['Date'] = pd.to_datetime(roll_df['Date'])
print(f"\nResult: {len(roll_df)} time points from {roll_df['Date'].min().date()} to {roll_df['Date'].max().date()}")

## 8. Full-Sample FEVD Table

Before looking at time-varying results, let's see the **average** spillover structure
over the entire sample. This gives us the "big picture" of who affects whom.

**How to read the table:**
- Row = the *receiver* (affected country)
- Column = the *transmitter* (source of shock)
- Cell value = % of the receiver's forecast error variance explained by the transmitter's shocks
- Diagonal = "own" effects
- Off-diagonal = spillovers

In [ ]:
# Estimate a single VAR on the FULL sample
full_sample = yield_changes[ALL_COUNTRIES].dropna()
model = VAR(full_sample)
var_full = model.fit(VAR_LAGS)

# Compute GFEVD
theta_full = generalized_fevd(var_full, H=FORECAST_HORIZON)
indices_full = compute_spillover_indices(theta_full, ALL_COUNTRIES)

# Display as a formatted table
fevd_table = pd.DataFrame(theta_full, index=ALL_COUNTRIES, columns=ALL_COUNTRIES).round(1)
print("Full-Sample Generalized FEVD Matrix (%):")
print("Rows = receiver, Columns = transmitter\n")
print(fevd_table.to_string())
print(f"\nTotal Spillover Index: {indices_full['total_spillover']:.1f}%")

# Heatmap
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(theta_full, cmap='YlOrRd', vmin=0, vmax=30)
for i in range(len(ALL_COUNTRIES)):
    for j in range(len(ALL_COUNTRIES)):
        color = 'white' if theta_full[i,j] > 18 else 'black'
        ax.text(j, i, f'{theta_full[i,j]:.1f}', ha='center', va='center', fontsize=9, color=color)
ax.set_xticks(range(len(ALL_COUNTRIES)))
ax.set_yticks(range(len(ALL_COUNTRIES)))
ax.set_xticklabels(ALL_COUNTRIES)
ax.set_yticklabels(ALL_COUNTRIES)
ax.set_xlabel('Transmitter (shock origin)', fontsize=11)
ax.set_ylabel('Receiver (affected market)', fontsize=11)
ax.set_title('Full-Sample GFEVD Matrix', fontweight='bold')
plt.colorbar(im, label='Spillover (%)')
plt.tight_layout()
plt.show()

## 9. Total Spillover Index Over Time

The **Total Spillover Index** captures overall market interconnectedness.
- Higher values = bond markets are more tightly linked (shocks propagate more)
- Lower values = markets are more segmented

Key patterns to look for:
- **Rising trend** during EMU convergence (late 1990s) and global tightening (2022+)
- **Sharp drops** during the Euro crisis (2010–2012) — markets fragmented
- **Spikes** during global events (GFC 2008, COVID 2020)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(roll_df['Date'], roll_df['total_spillover'], color='#003299', linewidth=1.2)
ax.fill_between(roll_df['Date'], roll_df['total_spillover'], alpha=0.3, color='#003299')
ax.set_xlabel('Date')
ax.set_ylabel('Total Spillover Index (%)')
ax.set_title('Diebold-Yilmaz Total Spillover Index (Generalized FEVD)', fontweight='bold')
ax.grid(alpha=0.3)

# Print current level
latest = roll_df.iloc[-1]
print(f"Latest total spillover: {latest['total_spillover']:.1f}% ({latest['Date'].date()})")
print(f"Sample average: {roll_df['total_spillover'].mean():.1f}%")
print(f"Sample max: {roll_df['total_spillover'].max():.1f}%")
print(f"Sample min: {roll_df['total_spillover'].min():.1f}%")

plt.tight_layout()
plt.show()

## 10. Net Spillovers by Country

**Net spillover = TO others − FROM others.**
- Positive = the country is a **net transmitter** (its shocks affect others more than it is affected)
- Negative = the country is a **net receiver** (it absorbs shocks from others)

This tells us which countries are "driving" bond markets and which are "following".

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 9), sharex=True)

# External countries
for c in EXTERNAL:
    axes[0].plot(roll_df['Date'], roll_df[f'{c}_net'], color=COLORS[c], linewidth=1.2, label=c)
axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_ylabel('Net Spillover')
axes[0].set_title('Net Spillovers: External Countries (+ = net transmitter)', fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(alpha=0.3)

# EA countries
for c in EA:
    axes[1].plot(roll_df['Date'], roll_df[f'{c}_net'], color=COLORS[c], linewidth=1.2, label=c)
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Net Spillover')
axes[1].set_title('Net Spillovers: Euro Area Countries (+ = net transmitter)', fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table of average net positions
print("\nAverage net spillover positions (full sample):")
for c in ALL_COUNTRIES:
    avg = roll_df[f'{c}_net'].mean()
    role = "NET TRANSMITTER" if avg > 0 else "net receiver"
    print(f"  {c:3s}: {avg:+.1f}  ({role})")

## 11. External vs Internal Spillovers to the Euro Area

This section answers: **"Are EA bond markets driven more by shocks from outside (US, UK, Japan)
or from within the EA itself?"**

### How the averages are computed

We compute the **average bilateral spillover** across all relevant pairs:

- **External → EA (average):** For each of the 3 external × 4 EA = 12 pairs, take $\Theta_{ij}$, then average. This answers: *"On average, how much does a single external country's shock explain of a single EA country's variance?"*

- **Within EA (average):** For each of the 4 × 3 = 12 intra-EA pairs, take $\Theta_{ij}$, then average. Same interpretation but for intra-EA shocks.

- **Individual source (e.g. US → EA):** Average US → DE, US → FR, US → IT, US → ES. This answers: *"On average, how much does a US shock explain of an EA country's variance?"*

**Important note:** These are *averages per pair*, not totals. To get TOTAL external influence on a given EA country, you would sum across external sources (e.g., US→DE + JP→DE + UK→DE). We show both below.

In [ ]:
# Compute external vs internal spillovers
ext_to_ea_cols = [f'{ext}_to_{ea}' for ea in EA for ext in EXTERNAL if f'{ext}_to_{ea}' in roll_df.columns]
int_ea_cols = [f'{ea1}_to_{ea2}' for ea2 in EA for ea1 in EA if ea1 != ea2 and f'{ea1}_to_{ea2}' in roll_df.columns]

# Average per pair
external_avg = roll_df[ext_to_ea_cols].mean(axis=1)
internal_avg = roll_df[int_ea_cols].mean(axis=1)

# Individual external sources → EA average
us_to_ea = roll_df[[f'US_to_{ea}' for ea in EA if f'US_to_{ea}' in roll_df.columns]].mean(axis=1)
jp_to_ea = roll_df[[f'JP_to_{ea}' for ea in EA if f'JP_to_{ea}' in roll_df.columns]].mean(axis=1)
uk_to_ea = roll_df[[f'UK_to_{ea}' for ea in EA if f'UK_to_{ea}' in roll_df.columns]].mean(axis=1)

# Total external spillover TO each EA country (summed, not averaged)
for ea in EA:
    ext_cols = [f'{ext}_to_{ea}' for ext in EXTERNAL if f'{ext}_to_{ea}' in roll_df.columns]
    int_cols = [f'{ea2}_to_{ea}' for ea2 in EA if ea2 != ea and f'{ea2}_to_{ea}' in roll_df.columns]
    print(f"{ea}: avg total external = {roll_df[ext_cols].sum(axis=1).mean():.1f}% "
          f"(sum of {len(ext_cols)} sources), "
          f"avg total internal = {roll_df[int_cols].sum(axis=1).mean():.1f}% "
          f"(sum of {len(int_cols)} sources)")

# Plot: Average per pair comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(roll_df['Date'], internal_avg, color='#222222', linewidth=1.5, label='Within EA (avg per pair)')
axes[0].plot(roll_df['Date'], external_avg, color='#999999', linewidth=1.5, label='External → EA (avg per pair)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Average Bilateral Spillover (%)')
axes[0].set_title('External vs Internal (Average Per Pair)', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot: Individual external sources
axes[1].plot(roll_df['Date'], us_to_ea, color=COLORS['US'], linewidth=1.2, label='US → EA')
axes[1].plot(roll_df['Date'], jp_to_ea, color=COLORS['JP'], linewidth=1.2, label='JP → EA')
axes[1].plot(roll_df['Date'], uk_to_ea, color=COLORS['UK'], linewidth=1.2, label='UK → EA')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Average Spillover to EA (%)')
axes[1].set_title('External Spillovers by Source Country', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Core–Periphery Dynamics

Within the Euro Area, we split countries into **core** (Germany, France) and
**periphery** (Italy, Spain).

This matters because:
- During crises, core and periphery can **decouple** (fragmentation)
- During normal times, spillovers flow in both directions (integration)
- The **direction** of spillovers reveals power dynamics

In [ ]:
# Core → Periphery spillovers
c2p_cols = [f'{core}_to_{periph}' for core in EA_CORE for periph in EA_PERIPHERY 
            if f'{core}_to_{periph}' in roll_df.columns]
p2c_cols = [f'{periph}_to_{core}' for periph in EA_PERIPHERY for core in EA_CORE 
            if f'{periph}_to_{core}' in roll_df.columns]

core_to_periph = roll_df[c2p_cols].mean(axis=1)
periph_to_core = roll_df[p2c_cols].mean(axis=1)
periph_net = periph_to_core - core_to_periph

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(roll_df['Date'], core_to_periph, color=COLORS['DE'], linewidth=1.2, label='Core → Periphery')
axes[0].plot(roll_df['Date'], periph_to_core, color=COLORS['IT'], linewidth=1.2, label='Periphery → Core')
axes[0].set_ylabel('Average Spillover (%)')
axes[0].set_title('Core–Periphery Spillover Flows', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].fill_between(roll_df['Date'], periph_net, where=periph_net >= 0, color=COLORS['IT'], alpha=0.5, label='Periphery net transmitter')
axes[1].fill_between(roll_df['Date'], periph_net, where=periph_net < 0, color=COLORS['DE'], alpha=0.5, label='Core net transmitter')
axes[1].plot(roll_df['Date'], periph_net, color='black', linewidth=0.8)
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Net Spillover')
axes[1].set_title('Periphery Net Position vs Core (+ = periphery transmits more)', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Bilateral Spillover Matrices by Regime

Here we compute the average FEVD matrix for specific historical periods.
This shows how the *structure* of spillovers changes across regimes.

**Regimes defined:**
- **Euro Crisis (2010–2012):** Sovereign debt crisis, fragmentation
- **QE Era (2015–2019):** ECB QE, low rates, convergence
- **Tightening (2022–2025):** Post-COVID hiking cycle

In [ ]:
regimes = {
    'Full Sample': (roll_df['Date'].min(), roll_df['Date'].max()),
    'Euro Crisis (2010-2012)': (pd.Timestamp('2010-01-01'), pd.Timestamp('2012-12-31')),
    'QE Era (2015-2019)': (pd.Timestamp('2015-01-01'), pd.Timestamp('2019-12-31')),
    'Tightening (2022-2025)': (pd.Timestamp('2022-01-01'), pd.Timestamp('2025-12-31')),
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (name, (start, end)) in enumerate(regimes.items()):
    # Get period data and estimate VAR
    mask = (yield_changes['Date'] >= start) & (yield_changes['Date'] <= end)
    period_data = yield_changes[mask][ALL_COUNTRIES].dropna()
    
    if len(period_data) < 100:
        axes[idx].text(0.5, 0.5, 'Insufficient data', ha='center', va='center')
        continue
    
    model = VAR(period_data)
    var_result = model.fit(VAR_LAGS)
    theta = generalized_fevd(var_result, H=FORECAST_HORIZON)
    
    im = axes[idx].imshow(theta, cmap='YlOrRd', aspect='auto', vmin=0, vmax=25)
    for i in range(len(ALL_COUNTRIES)):
        for j in range(len(ALL_COUNTRIES)):
            if i != j:
                color = 'white' if theta[i,j] > 15 else 'black'
                axes[idx].text(j, i, f'{theta[i,j]:.1f}', ha='center', va='center', fontsize=7, color=color)
    
    axes[idx].set_xticks(range(len(ALL_COUNTRIES)))
    axes[idx].set_yticks(range(len(ALL_COUNTRIES)))
    axes[idx].set_xticklabels(ALL_COUNTRIES, fontsize=8)
    axes[idx].set_yticklabels(ALL_COUNTRIES, fontsize=8)
    axes[idx].set_title(name, fontsize=10, fontweight='bold')

fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='Spillover (%)')
fig.suptitle('Bilateral Spillover Matrices by Regime', fontsize=13, fontweight='bold', y=0.98)
plt.show()

## 14. Regime Comparison Summary

Average spillover measures across different policy regimes.

In [ ]:
regime_defs = {
    'Pre-Draghi (2010-2012)': ('2010-01-01', '2012-07-25'),
    'QE Era (2015-2019)': ('2015-01-01', '2019-12-31'),
    'COVID/PEPP (2020-2021)': ('2020-01-01', '2021-12-31'),
    'Tightening (2022-2023)': ('2022-01-01', '2023-12-31'),
    'Cutting Cycle (2024+)': ('2024-01-01', '2025-12-31'),
}

rows = []
for name, (start, end) in regime_defs.items():
    mask = (roll_df['Date'] >= start) & (roll_df['Date'] <= end)
    rd = roll_df[mask]
    if len(rd) == 0:
        continue
    row = {'Regime': name, 'N windows': len(rd), 'Total Spillover': rd['total_spillover'].mean()}
    for c in ALL_COUNTRIES:
        row[f'{c} net'] = rd[f'{c}_net'].mean()
    rows.append(row)

regime_table = pd.DataFrame(rows).set_index('Regime')
print(regime_table.round(1).to_string())

# Bar chart
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(regime_table.index, regime_table['Total Spillover'], color='#003299', alpha=0.8)
ax.set_ylabel('Total Spillover Index (%)')
ax.set_title('Average Total Spillover by Regime', fontweight='bold')
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=30, ha='right')
for i, v in enumerate(regime_table['Total Spillover']):
    ax.text(i, v + 0.3, f'{v:.1f}', ha='center', fontsize=10)
plt.tight_layout()
plt.show()

## 15. Key Findings

The analysis above typically reveals several patterns (verify with your own output):

1. **Spillover intensity has increased** — bond markets are more interconnected now than in the 1990s
2. **The US is the dominant external transmitter** to EA bond markets
3. **Internal EA spillovers exceed external ones** — EA countries affect each other more than outsiders affect them
4. **Germany and France are the key transmitters** within the EA; Italy and Spain are net receivers
5. **The Euro crisis (2010–2012) saw fragmentation** — total spillovers dropped as core and periphery decoupled
6. **Japan has decoupled post-YCC** — JP→EA spillovers fell sharply after the Bank of Japan ended yield curve control
7. **France acts as a pivot** between core and periphery, transmitting shocks in both directions

## 16. Export Results

Save all outputs to CSV and Excel for further analysis or sharing.

In [ ]:
from pathlib import Path
out_dir = Path("notebook_output")
out_dir.mkdir(exist_ok=True)

# Save rolling spillovers
roll_df.to_csv(out_dir / "rolling_spillovers.csv", index=False)
print(f"Saved: rolling_spillovers.csv ({len(roll_df)} rows)")

# Save full-sample FEVD table
fevd_table.to_csv(out_dir / "full_sample_fevd.csv")
print(f"Saved: full_sample_fevd.csv")

# Save regime statistics
regime_table.to_csv(out_dir / "regime_statistics.csv")
print(f"Saved: regime_statistics.csv")

# Save monthly external spillovers to US (Excel)
roll_monthly = roll_df.copy()
roll_monthly['YearMonth'] = roll_monthly['Date'].dt.to_period('M')

ext_to_us_cols = [c for c in roll_df.columns if c.endswith('_to_US')]
monthly_agg = roll_monthly.groupby('YearMonth')[ext_to_us_cols].mean()
monthly_agg['Total_External_to_US'] = monthly_agg.sum(axis=1)
monthly_agg.index = monthly_agg.index.to_timestamp()

with pd.ExcelWriter(out_dir / "external_spillovers_to_us_monthly.xlsx", engine='openpyxl') as writer:
    monthly_agg.to_excel(writer, sheet_name='Monthly External to US')
print(f"Saved: external_spillovers_to_us_monthly.xlsx")

print(f"\nAll outputs in: {out_dir.resolve()}")

## Appendix: How to Modify This Analysis

| I want to... | Change this... | In Section... |
|--------------|---------------|---------------|
| Use different countries | Edit `EXTERNAL`, `EA_CORE`, `EA_PERIPHERY` lists | Section 2 |
| Use a different CSV file | Change `DATA_PATH` | Section 2 |
| Use more/fewer VAR lags | Change `VAR_LAGS` | Section 2 |
| Change the forecast horizon | Change `FORECAST_HORIZON` | Section 2 |
| Make smoother/choppier time series | Increase/decrease `ROLLING_WINDOW` | Section 2 |
| Speed up / slow down computation | Increase/decrease `STEP_SIZE` | Section 2 |
| Add new regime periods | Edit `regime_defs` dict | Section 14 |
| Change chart colours | Edit `COLORS` dict | Section 2 |
| Use yield levels instead of changes | Remove the `.diff()` in Section 4 (NOT recommended) | Section 4 |
| Use Cholesky instead of Generalized FEVD | Replace `generalized_fevd()` with statsmodels built-in `fevd()` | Section 6 |

**After changing parameters**, re-run all cells from Section 7 onwards (Kernel → Restart & Run All).